In [46]:
import pyhdb
import yaml
import pandas as pd
from sklearn import linear_model
import numpy as np

In [4]:
with open('../credentials.yml', 'r') as f:
    credentials = yaml.load(f)

In [5]:
connection = pyhdb.connect(**credentials)
cursor = connection.cursor()

In [10]:
query = '''
select distinct h.category, 2012 - p.yearofbirth age, case when p.gender = 'F' then 0 else 1 end gender,
            ts.bmi, ts.systolicbp, ts.diastolicbp
from "Patient" p, "PatientSmokingStatus" pss, smokingstatushelper h, (
    select avg(bmi) bmi, avg(systolicbp) systolicbp, avg(diastolicbp) diastolicbp, patientguid
    from "Transcript" where bmi != 0 and systolicbp != 0 and diastolicbp != 0 group by patientguid
) ts
where p.patientguid=pss.patientguid and h.smokingstatusguid=pss.smokingstatusguid and ts.patientguid=p.patientguid
'''
cursor.execute(query)
result = cursor.fetchall()

In [11]:
print('result size: ' + str(len(result)))

result size: 4504


In [72]:
def predict(model_s, model_d, patient_data):
    result_s = model_s.predict([patient_data])
    result_d = model_d.predict([patient_data])
    print('Predicted blood pressure for patient ...{}{}'.format(result_s, result_d))

def prepare_data(data, blood_pressure):
    new_data = [[x[0], x[2], x[3], x[4]] for x in data]
    
    if blood_pressure == 'systolic':
        return new_data, [round(x[4]) for x in data]
    elif blood_pressure == 'diastolic':
        return new_data, [round(x[5]) for x in data]
        
x, y = prepare_data(result, 'systolic')
model_s = linear_model.LogisticRegression()
model_s.fit(x, y)

x, y = prepare_data(result, 'diastolic')
model_d = linear_model.LogisticRegression()
model_d.fit(x, y)

predict(model_s, model_d, [6, 50, 0, 23])
    

Predicted blood pressure for patient ...[157][100]
